## Feature Engineering

#### import data

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
df = pd.read_csv('data/merged_df.csv', sep=",", index_col=0)

In [25]:
df.head()

,url,title,date,author,description,text,Article Length,sentiment,sentiment_label,Stock,Open,Low,Close,Adjusted_Close,Volume,Month,Price_Change_Pct,Adjusted_Close_Change
0,https://www.cnbc.com/2015/02/24/apples-record-...,Apple’s record rally depends on Obama?,2015-02-24,John Melloy,The size of Apple's capital return program wil...,In order for Apple to extend its 21 percent ra...,499.0,1.0,very negative,MSFT,44.299999,43.919998,44.090000,38.414326,25271700.0,2.0,-0.135903,-0.001359
1,https://www.cnbc.com/2015/02/25/dockers-rise-f...,Docker's rise from sleeper to open source king,2015-02-25,Ari Levy,Docker spent 2014 partnering with the world's ...,"In December 2013, Google announced that its c...",777.0,1.0,very negative,MSFT,44.090000,43.799999,43.990002,38.327202,29759800.0,2.0,-0.226805,-0.002268
2,https://www.cnbc.com/2015/03/02/apple-value-to...,Apple value to hit $1 trillion in 1 year,2015-03-02,John Melloy,Apple will rise to $1 trillion in market value...,Apple will rise to $1 trillion in market value...,592.0,2.0,negative,MSFT,44.189999,43.549999,43.880001,38.231373,31924000.0,3.0,0.068421,0.000684
3,https://www.cnbc.com/2015/03/05/is-apple-world...,"Is Apple, world’s largest stock, ‘underowned’?",2015-03-06,John Melloy,A top Wall Street strategist makes the case th...,"Jason Trennert, a top Wall Street strategist, ...",1112.0,4.0,positive,MSFT,43.110001,42.150002,42.360001,36.907036,36248800.0,3.0,-1.739736,-0.017397
4,https://www.cnbc.com/2015/03/09/apple-may-sell...,Apple may sell 1 billion 'life-saving' watches,2015-03-09,John Melloy,CNBC Pro Preview: Influential technology analy...,The following is a free preview of the content...,506.0,1.0,very negative,MSFT,43.130001,42.189999,42.849998,37.333958,32108000.0,3.0,1.156747,0.011567


#### 7 day moving average

In [26]:
#  Moving Averages for the Closing Prices of each stock
df['30_day_MA'] = df.groupby('Stock')['Close'].transform(lambda x: x.rolling(window=30).mean())
df['60_day_MA'] = df.groupby('Stock')['Close'].transform(lambda x: x.rolling(window=60).mean())
df['90_day_MA'] = df.groupby('Stock')['Close'].transform(lambda x: x.rolling(window=90).mean())

In [27]:
# Moving Averages
df['SMA_30'] = df.groupby('Stock')['Close'].transform(lambda x: x.rolling(window=30).mean())
df['SMA_60'] = df.groupby('Stock')['Close'].transform(lambda x: x.rolling(window=60).mean())
df['SMA_90'] = df.groupby('Stock')['Close'].transform(lambda x: x.rolling(window=90).mean())

df['EMA_30'] = df.groupby('Stock')['Close'].transform(lambda x: x.ewm(span=30, adjust=False).mean())
df['EMA_60'] = df.groupby('Stock')['Close'].transform(lambda x: x.ewm(span=60, adjust=False).mean())
df['EMA_90'] = df.groupby('Stock')['Close'].transform(lambda x: x.ewm(span=90, adjust=False).mean())

# RSI Calculation
def calculate_rsi(x, periods=14):
    delta = x.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df['RSI'] = df.groupby('Stock')['Close'].transform(calculate_rsi)

# MACD Calculation
df['EMA_12'] = df.groupby('Stock')['Close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
df['EMA_26'] = df.groupby('Stock')['Close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
df['MACD'] = df['EMA_12'] - df['EMA_26']
df['Signal_Line'] = df.groupby('Stock')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean())
df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']

In [28]:
df.columns

Index(['url', 'title', 'date', 'author', 'description', 'text',
       'Article Length', 'sentiment', 'sentiment_label', 'Stock', 'Open',
       'Low', 'Close', 'Adjusted_Close', 'Volume', 'Month', 'Price_Change_Pct',
       'Adjusted_Close_Change', '30_day_MA', '60_day_MA', '90_day_MA',
       'SMA_30', 'SMA_60', 'SMA_90', 'EMA_30', 'EMA_60', 'EMA_90', 'RSI',
       'EMA_12', 'EMA_26', 'MACD', 'Signal_Line', 'MACD_Histogram'],
      dtype='object')

In [29]:
def nulls_summary_table(df):
    """
    Returns a summary table showing null value counts and percentage

    Parameters:
    df (DataFrame): Dataframe to check

    Returns:
    
    null_values (DataFrame)
    """
    null_values = pd.DataFrame(df.isnull().sum())
    null_values[1] = null_values[0]/len(df)
    null_values.columns = ['null_count','null_pct']
    return null_values

nulls_summary_table(df)

,null_count,null_pct
url,0,0.000000
title,0,0.000000
date,0,0.000000
author,0,0.000000
description,0,0.000000
text,0,0.000000
Article Length,0,0.000000
sentiment,0,0.000000
sentiment_label,0,0.000000
Stock,0,0.000000


In [30]:
df =df[[ 'date', 'Stock', 'Open', 'Low', 'Close', 'Volume',"30_day_MA","60_day_MA","90_day_MA",  'SMA_30', 'SMA_60', 'SMA_90', 'EMA_30', 'EMA_60', 'EMA_90', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'Signal_Line', 'MACD_Histogram','Price_Change_Pct','Article Length', 'sentiment']]

df.rename(columns={'sentiment': 'article_sentiment'}, inplace=True)

df = df.dropna()
df.reset_index(drop=True, inplace=True)

In [31]:
df.head()

,date,Stock,Open,Low,Close,Volume,30_day_MA,60_day_MA,90_day_MA,SMA_30,...,EMA_90,RSI,EMA_12,EMA_26,MACD,Signal_Line,MACD_Histogram,Price_Change_Pct,Article Length,article_sentiment
0,2016-07-20,MSFT,56.840000,55.529999,55.910000,89893300.0,51.945666,52.309333,50.519333,51.945666,...,50.523063,71.721804,52.735685,52.177104,0.558581,0.040408,0.518174,5.311734,1394.0,4.0
1,2016-07-25,MSFT,56.740002,56.259998,56.730000,25610600.0,52.099999,52.403333,50.659777,52.099999,...,50.659480,74.606512,53.350195,52.514356,0.835840,0.199494,0.636346,0.282835,623.0,4.0
2,2016-08-01,MSFT,56.750000,56.139999,56.580002,26003400.0,52.249333,52.476499,50.799666,52.249333,...,50.789601,91.388238,53.847088,52.815515,1.031574,0.365910,0.665664,-0.176426,269.0,4.0
3,2016-08-03,MSFT,57.110001,56.490002,56.970001,22075600.0,52.411666,52.548499,50.945111,52.411666,...,50.925434,91.799325,54.327537,53.123254,1.204282,0.533585,0.670698,0.689288,904.0,4.0
4,2016-08-05,MSFT,58.209999,57.450001,57.959999,29335200.0,52.606999,52.658499,51.118444,52.606999,...,51.080040,92.480413,54.886377,53.481532,1.404845,0.707837,0.697009,0.993204,679.0,1.0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4983 entries, 0 to 4982
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               4983 non-null   object 
 1   Stock              4983 non-null   object 
 2   Open               4983 non-null   float64
 3   Low                4983 non-null   float64
 4   Close              4983 non-null   float64
 5   Volume             4983 non-null   float64
 6   30_day_MA          4983 non-null   float64
 7   60_day_MA          4983 non-null   float64
 8   90_day_MA          4983 non-null   float64
 9   SMA_30             4983 non-null   float64
 10  SMA_60             4983 non-null   float64
 11  SMA_90             4983 non-null   float64
 12  EMA_30             4983 non-null   float64
 13  EMA_60             4983 non-null   float64
 14  EMA_90             4983 non-null   float64
 15  RSI                4983 non-null   float64
 16  EMA_12             4983 

For regreesion subtract, 3 for each sentmient value so the range for the sentiment values is -2 to 2 

In [33]:
df['article_sentiment'] = df['article_sentiment'] - 3

In [34]:
df.to_csv('data/df_fe.csv')